In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.0 MB/s 
     |████████████████████████████████| 588.3 MB 21 kB/s 
     |████████████████████████████████| 6.0 MB 69.1 MB/s 
     |████████████████████████████████| 439 kB 64.2 MB/s 
     |████████████████████████████████| 1.7 MB 51.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
16252,"average, staff attitude stayed convenience fer...",0
18564,worth visiting just spent week resort 16/3 24/...,1
17792,"pleasure write review, let start saying did no...",1
19044,"charming treasure stayed hotel week, lovely qu...",1
19729,nice stay walkable las ramblas nice stay clean...,1
19987,chic haven el born having really bad experienc...,1
6062,new years punta cana wife looking minute getaw...,0
6647,hope n't hope n't like 90 degree weather b/c d...,0
17951,wonderful place kids just got oceans sand say ...,1
4280,"hurry leave, really sucks, terrible food cold,...",0


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
2559,poor quality attended company convention ec 24...,0
582,light dark bali just got bali mandira great li...,1
3003,caution bavaro princess writing letter inform ...,0
501,"loved hotel, pleasant surprise booked travel o...",1
389,princess pea arrived hour check-in waiting 15 ...,0
1508,"okay not fantastic, checking price upscale hot...",0
3090,just ok.. not 5 star.. group 15 stayed melia c...,0
3875,expectations low wife spent 5 days natura park...,0
1027,"thanks tripadvisor, visited march weather perf...",1
227,fantastic hotel fabulous stay sheraton downtow...,1


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
lambert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert


In [ ]:
lambert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/lambert_en_uncased_L-24_H-1024_A-16/2", trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = lambert_preprocess(text_input)
outputs = lambert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=5) # five epochs

Epoch 1/5
49/49 [==============================] - 2169s 44s/step - loss: 0.6072 - accuracy: 0.6615 - precision: 0.6667 - recall: 0.6021
Epoch 2/5
49/49 [==============================] - 2153s 44s/step - loss: 0.4693 - accuracy: 0.8054 - precision: 0.8217 - recall: 0.7639
Epoch 3/5
49/49 [==============================] - 2075s 42s/step - loss: 0.4049 - accuracy: 0.8324 - precision: 0.8345 - recall: 0.8156
Epoch 4/5
49/49 [==============================] - 2097s 43s/step - loss: 0.3762 - accuracy: 0.8516 - precision: 0.8558 - recall: 0.8342
Epoch 5/5
49/49 [==============================] - 2098s 43s/step - loss: 0.3393 - accuracy: 0.8715 - precision: 0.8912 - recall: 0.8369


In [30]:
y_predicted = model.predict(X_test)
y_predicted


146/146 [==============================] - 6304s 43s/step


array([[0.03277574],
       [0.10036262],
       [0.07932568],
       ...,
       [0.5471033 ],
       [0.7062275 ],
       [0.9875999 ]], dtype=float32)

In [31]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [32]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [33]:
accuracy_score(y_test, y_predicted)

0.792763862128024

In [34]:
precision_score(y_test, y_predicted)


0.8405263157894737

In [35]:
recall_score(y_test, y_predicted)

0.7060123784261715

In [37]:
f1_score(y_test, y_predicted)

0.7674195098510331

In [38]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [39]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted)


In [40]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/lambert_trained_on_hotel_reviews_29_december'

In [41]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/lambert_trained_on_hotel_reviews_29_december


In [42]:
model.save(saved_model_path, include_optimizer=True) 

In [43]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.792764,0.840526,0.706012,0.76742
